# Adding Dependeces

In [4]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

In [5]:
parameters = Model(
    Cubic(
        μ₁        = 0.0001:0.0001:0.0001,
        rₘᵢₙ      = 2.0:0.1:2.0,
        rₘₐₓ      = 3.0:0.1:3.0
    ),
    Contractile(
        fₚ        = 0.001:0.001:0.001
    ),
    Time(
        t_f       = 100000, 
        dt        = 0.1
    ),
    Neighbor(
        n_knn     = 100,
        nn        = 100
    ),
    Geometry(
        R_agg     = 9,
        position  = [
            1.0    0.0    0.0;
            0.0    1.0    0.0
        ]
    ),
    Simulation(
        n_text    = 200,
        path      = "../../results",
        name_cell = "CHO"
    )
)

Model(Cubic{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}(0.0001:0.0001:0.0001, 2.0:0.1:2.0, 3.0:0.1:3.0), Contractile{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}(0.001:0.001:0.001), Time(100000.0, 0.1), Neighbor(100, 100), Geometry(9.0, [1.0 0.0 0.0; 0.0 1.0 0.0]), Simulation(200, "../../results", "CHO"))

## Initial Conditions

In [8]:
function run_all(parameters)
    # Testing all the conditions of the fusion
    iter = vcat(
        [eval(:(parameters.Contractile.$s)) for s in fieldnames(Contractile)],
        [eval(:(parameters.Force.$s)) for s in fieldnames(typeof(parameters.Force))]
    )
    for k in Iterators.product(iter...)
        # Iterated Parameters
        it_contractile = Contractile(k[1])
        it_force = Cubic(k[2:end]...)
        # force(r) =  force_func(it_force::typeof(it_force), r)
        force(r) =  force_func(it_force, r)
        
        # Initial Data
        global Agg = (
            Aggregate(
                Neighbor(50, 12), 
                PositionCell(
                    # Loading Data
                    Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
                )
            )
        )

        # Generating the Path
        if size(parameters.Geometry.position, 1) == 1
            NumAgg = "OneAgg"
        else
            NumAgg = "Fusion"
        end
        name_force = ["$(fieldnames(typeof(parameters.Force))[i])_$(eval(:($it_force.$(fieldnames(typeof(parameters.Force))[i]))))" for i in 1:3]
        name_contractile = ["fₚ_$(it_contractile.fₚ)"]
        Path = join(
            [
                parameters.Simulation.path, 
                parameters.Simulation.name_cell, 
                NumAgg,
                "R_Agg_$(parameters.Geometry.R_agg)",
                "T_$(parameters.Time.t_f)_dT_$(parameters.Time.dt)",
                "nn_$(parameters.Neighbor.nn)_t_knn_$(parameters.Neighbor.n_knn)",
                join(name_force[end-1:end], "_"),
                join(name_force[1:end-2], "_")
            ], 
            "/"
        )
        File = join(
            [
                "R_Agg_$(parameters.Geometry.R_agg)",
                "T_$(parameters.Time.t_f)_dT_$(parameters.Time.dt)",
                "nn_$(parameters.Neighbor.nn)_t_knn_$(parameters.Neighbor.n_knn)",
                join(name_force[end-1:end], "_"),
                join([name_contractile[1], name_force[1:end-2][1]], "_")
            ], 
            "|"
        )*".xyz"
        
        for i in size(split(Path,"/"),1)-6:size(split(Path,"/"),1)
            if !isdir(join(split(Path,"/")[1:i], "/"))
                mkdir(join(split(Path,"/")[1:i], "/"))
            end
        end

        # Calculating all above
        println("---------------------------------------------------------------")
        println("Calculating "*File*"\nfor $(size(parameters.Geometry.position, 1)) Aggregate(s)")

        # if File in readdir(Path)
        #     if countlines(Path*"/"*File) < (2*size(Agg.Position.X,1)+2)*(parameters.Simulation.n_text+1)
        #         println("Calculated with less data. Recalculating")
        #         rm(Path*"/"*File)
        #         if size(parameters.Geometry.position, 1) == 1
        #             one_aggregate(Path*"/"*File, true, parameters, it_contractile, Agg)
        #         else
        #             fusion(Path*"/"*File, true, parameters, it_contractile, Agg)
        #         end
        #     else
        #         println("This is already calculated")           
        #     end
        # else
            if size(parameters.Geometry.position, 1) == 1
                one_aggregate(Path*"/"*File, true, parameters, it_contractile, Agg)
            else
                fusion(Path*"/"*File, true, parameters, it_contractile, Agg)
            end
        # end
        
    end
end
run_all(parameters)

LoadError: syntax: Global method definition around In[8]:12 needs to be placed at the top level, or use "eval".